# Import

In [ ]:
import pandas as pd

df_drug = pd.read_csv('dataset/train.csv',sep=',')

In [4]:
df_drug

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782
...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89,6.963020
32161,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,0.899076
32162,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25,6.241812
32163,45410,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8,21-Jan-15,22,7.940428


In [6]:
df_drug.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score'],
      dtype='object')

In [8]:
df_drug.drop(['drug_approved_by_UIC','number_of_times_prescribed','base_score','effectiveness_rating','patient_id'],axis=1)

,name_of_drug,use_case_for_drug,review_by_patient
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati..."
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ..."
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh..."
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around..."
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard..."
...,...,...,...
32160,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now..."
32161,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ..."
32162,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting..."
32163,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now...."


In [21]:
len(df_drug['name_of_drug'].unique())


2220

In [22]:
len(df_drug['use_case_for_drug'].unique())


636

In [25]:
# Combine the columns into a single text column called 'combined_text'
df_drug['combined_text'] = df_drug.apply(
    lambda x: f"Drug: {x['name_of_drug']} | Use Case: {x['use_case_for_drug']} | Review: {x['review_by_patient']}",
    axis=1
)


In [26]:
df_drug

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,combined_text
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969,Drug: Valsartan | Use Case: Left Ventricular D...
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458,"Drug: Guanfacine | Use Case: ADHD | Review: ""M..."
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969,Drug: Lybrel | Use Case: Birth Control | Revie...
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176,Drug: Buprenorphine / naloxone | Use Case: Opi...
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782,Drug: Cialis | Use Case: Benign Prostatic Hype...
...,...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89,6.963020,"Drug: Cymbalta | Use Case: Anxiety | Review: ""..."
32161,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,0.899076,Drug: Nexplanon | Use Case: Birth Control | Re...
32162,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25,6.241812,Drug: Venlafaxine | Use Case: Panic Disorde | ...
32163,45410,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8,21-Jan-15,22,7.940428,Drug: Fluoxetine | Use Case: Obsessive Compuls...


# Embedding

In [27]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for the 'combined_text' column
df_drug['embedding'] = df_drug['combined_text'].apply(lambda x: model.encode(x))


In [28]:
# Display the first few rows of the combined text and embeddings
print(df_drug[['combined_text', 'embedding']].head())


                                       combined_text  \
0  Drug: Valsartan | Use Case: Left Ventricular D...   
1  Drug: Guanfacine | Use Case: ADHD | Review: "M...   
2  Drug: Lybrel | Use Case: Birth Control | Revie...   
3  Drug: Buprenorphine / naloxone | Use Case: Opi...   
4  Drug: Cialis | Use Case: Benign Prostatic Hype...   

                                           embedding  
0  [0.034069546, 0.02242714, -0.021552138, -0.006...  
1  [-0.0073752627, -0.023261907, -0.0950454, -0.0...  
2  [-0.029566338, -0.08697223, -0.042140175, 0.02...  
3  [-0.012879459, -0.031134004, 0.013913328, 0.10...  
4  [-0.045360405, -0.08541006, 0.01331802, 0.0411...  


# VectorStore

In [ ]:
import numpy as np

# Convert the list of embeddings into a NumPy array
embedding_matrix = np.array(df_drug['embedding'].tolist())

# Get the embedding dimensions
embedding_dim = embedding_matrix.shape[1]
print(f"Embedding matrix shape: {embedding_matrix.shape}")


In [ ]:
import faiss

# Create an index with L2 (Euclidean distance)
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(embedding_matrix)

# Verify the number of embeddings added
print(f"Number of embeddings in the index: {index.ntotal}")
